In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [7]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 4, 11)
init_page = 1
init_offset = 0

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-04-13
page  1
https://index.sindonews.com/index/0/0?t=2018-04-13
https://international.sindonews.com/read/1297758/40/as-bakal-riview-kerja-sama-dengan-indonesia-1523638418
Insert berita  AS BakalRiviewKerja Sama dengan Indonesia
salah2
https://metro.sindonews.com/read/1297756/170/peringati-isra-mikraj-di-monas-anies-memberikan-cahaya-di-ibu-kota-1523638006
Insert berita  Peringati Isra Mikraj di Monas, Anies: Memberikan Cahaya di Ibu Kota
salah2
https://metro.sindonews.com/read/1297755/171/berbasis-ti-menkumham-sebut-lapas-cibinong-sudah-bagus-1523637817
Insert berita  Berbasis TI, Menkumham Sebut Lapas Cibinong Sudah Bagus
salah2
https://daerah.sindonews.com/read/1297753/22/presiden-jokowi-besok-tutup-munas-alim-ulama-ppp-di-semarang-1523636723
Insert berita  Presiden Jokowi Besok Tutup Munas Alim Ulama PPP di Semarang
salah2
https://international.sindonews.com/read/1297754/41/rusia-mata-matai-skripal-dan-putrinya-selama-5-tahun-1523637287
Insert berita  Rusia Mata-matai Skripal 

masuk
https://daerah.sindonews.com/read/1297712/174/dukun-aborsi-tusuk-kelamin-mahasiswa-uin-hingga-rahim-dengan-kayu-1523625109
Insert berita  Dukun Aborsi Tusuk Kelamin Mahasiswa UIN hingga Rahim dengan Kayu
masuk
https://metro.sindonews.com/read/1297711/170/dibekuk-polisi-riza-shahab-saya-menyesal-gunakan-narkoba-1523625047
Insert berita  Dibekuk Polisi, Riza Shahab: Saya Menyesal Gunakan Narkoba
masuk
https://ekbis.sindonews.com/read/1297688/34/optimalisasi-alat-pertanian-upja-smart-mobile-siap-dilaunching-1523619587
Insert berita  Optimalisasi Alat Pertanian, UPJA Smart Mobile Siap Dilaunching
masuk
https://ekbis.sindonews.com/read/1297710/179/rei-dukung-pembentukan-kek-pariwisata-di-bangka-1523624814
Insert berita  REI Dukung Pembentukan KEK Pariwisata di Bangka
masuk
https://nasional.sindonews.com/read/1297708/15/perindo-penyakit-sosial-bisa-hilang-dengan-pemberdayaan-ekonomi-masyarakat-1523624767
Insert berita  Perindo: Penyakit Sosial Bisa Hilang dengan Pemberdayaan Ekonomi Ma

masuk
https://daerah.sindonews.com/read/1297667/174/mahasiswi-uin-yang-aborsi-ternyata-sudah-pacaran-selama-2-tahun-1523614524
Insert berita  Mahasiswi UIN yang Aborsi Ternyata Sudah Pacaran Selama 2 Tahun
masuk
https://lifestyle.sindonews.com/read/1297596/185/resep-lodeh-pepaya-muda-yang-menyehatkan-1523604305
Insert berita  Resep Lodeh Pepaya Muda yang Menyehatkan
masuk
https://ekbis.sindonews.com/read/1297666/33/agus-martowardojo-pertumbuhan-ekonomi-indonesia-terus-meningkat-1523613921
Insert berita  Agus Martowardojo: Pertumbuhan Ekonomi Indonesia Terus Meningkat
masuk
https://metro.sindonews.com/read/1297661/170/diduga-dianiaya-mahasiswa-unpam-laporkan-dosen-ke-polisi-1523613092
Insert berita  Diduga Dianiaya, Mahasiswa Unpam Laporkan Dosen ke Polisi
masuk
https://nasional.sindonews.com/read/1297665/12/repdem-siapkan-1-500-posko-pemenangan-jokowi-1523613852
Insert berita  Repdem Siapkan 1.500 Posko Pemenangan Jokowi
masuk
https://autotekno.sindonews.com/read/1297651/122/xiaomi-com

masuk
https://nasional.sindonews.com/read/1297615/15/agama-dan-budaya-adalah-ruh-pancasila-tak-perlu-dipertentangkan-1523607386
Insert berita  Agama dan Budaya adalah Ruh Pancasila, Tak Perlu Dipertentangkan
masuk
https://daerah.sindonews.com/read/1297625/174/3-harimau-sumatera-mengamuk-polisi-koordinasi-dengan-bbksda-1523608867
Insert berita  3 Harimau Sumatera Mengamuk, Polisi Koordinasi dengan BBKSDA
masuk
https://soccer.sindonews.com/read/1297614/59/kampiun-piala-asia-u23-siap-berlaga-di-stadion-pakansari-1523607335
Insert berita  Kampiun Piala Asia U23 Siap Berlaga di Stadion Pakansari
masuk
https://daerah.sindonews.com/read/1297613/174/ketua-dpd-pdip-riau-dicopot-1523607271
Insert berita  Ketua DPD PDIP Riau Dicopot
masuk
https://lifestyle.sindonews.com/read/1297587/185/subbet-cemilan-talas-dan-pisang-kepok-khas-mentawai-1523602625
Insert berita  Subbet, Cemilan Talas dan Pisang Kepok Khas Mentawai
masuk
https://metro.sindonews.com/read/1297612/171/terdapat-narkoba-sandiaga-pasti

masuk
https://ekbis.sindonews.com/read/1297571/34/dipasena-dinilai-layak-masuk-proyek-strategis-nasional-1523599138
Insert berita  Dipasena Dinilai Layak Masuk Proyek Strategis Nasional
masuk
https://sports.sindonews.com/read/1297570/53/seni-merajah-tubuh-pesepak-bola-dunia-1523598292
Insert berita  Seni Merajah Tubuh Pesepak Bola Dunia
masuk
https://lifestyle.sindonews.com/read/1297569/186/jo-malone-pencipta-parfum-ikonik-1523596767
Insert berita  Jo Malone Pencipta Parfum Ikonik
masuk
https://international.sindonews.com/read/1297568/40/myanmar-tak-siap-repatriasi-rohingya-1523596307
Insert berita  Myanmar Tak Siap Repatriasi Rohingya
masuk
page  18
https://index.sindonews.com/index/0/170?t=2018-04-13
https://lifestyle.sindonews.com/read/1297542/156/yuk-liburan-ke-wisata-air-terjun-sigandul-batang-1523590197
Insert berita  Yuk, Liburan ke Wisata Air Terjun Sigandul, Batang
masuk
https://ekbis.sindonews.com/read/1297562/34/tambahan-libur-lebaran-bakal-diputuskan-pekan-depan-1523594457
